In [4]:
from components.embeddings import HFembedding, OpenAIembedding
from components.data_chunker import ReadFile
from components.databases import VectorDB
import os
import json
from typing import Dict, List, Optional, Tuple, Union
import PyPDF2

In [12]:
filter=ReadFile('./data')
docs=filter.get_all_chunk_content(200,150)
docs

In [9]:
from huggingface_hub import login
login()

In [13]:
hf = HFembedding(path = "microsoft/Phi-3-mini-4k-instruct")

db = VectorDB(docs)
vectors = db.get_vector(hf)
db.persist()
db.load_vector()
res = db.query("What did Joe Biden do in June 2024",hf,3)
res

In [ ]:
q1 = "What happend to Joe Biden in July 2024?"
info1 = db.query(q1, hf, 3)

msg1 = [
    ("system", "Please refer to the following content: {}".info),
    ("human", q1),
]

chat.invoke(msg1).content

In [ ]:
q2 = "Who will replace Biden after withdrew his candidacy?"
info1 = db.query(q2, hf, 3)

msg2 = [
    ("system", "Please refer to the following content: {}".format(info)),
    ("human", q2),
]

chat.invoke(msg1).content

## Live Demo of Chat Bot

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat = ChatHuggingFace(llm=llm, verbose=True)

In [ ]:
import gradio as gr
import time


def echo(message, history):
    info = db.query(message,hf,3)
    messages = [
    ("system", "Please refer to the following content: {}".format(info)),
    ("human", message),
]
    result=chat.invoke(messages).content
    for i in range(len(result)):
        time.sleep(0.02)
        yield result[: i+1]



demo = gr.ChatInterface(fn=echo, 
                        examples=["What happend to Joe Biden in July 2024", 
                                  "Who will replace Biden after withdrew his candidacy?"], 
                        title="Chat Bot",
                        theme="soft")
demo.launch()